#### 고려해야 할 사항
- 

# Import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import mglearn

#### 한글 사용을 위한 method

In [2]:
matplotlib.rcParams['font.family']='Malgun Gothic'   # 한글 사용
matplotlib.rcParams['axes.unicode_minus'] = False

#### 분석에 필요한 각 파일 read

In [3]:
activity = pd.read_csv('train_activity.csv')
label = pd.read_csv('train_label.csv')
combat = pd.read_csv('train_combat.csv')
payment = pd.read_csv('train_payment.csv')
trade = pd.read_csv('train_trade.csv')
pledge = pd.read_csv('train_pledge.csv')

#### activity 파일 편집
- Char_id 와 Acc_id 기준으로 groupby.sum()
- 자동으로 합쳐지는 day를 삭제

In [4]:
activity_g = activity.groupby(['char_id','acc_id' ],as_index=False).sum() # groupby 한 다음에 index 분리
activity_g.drop(columns = 'day', inplace = True)
activity_g

,char_id,acc_id,playtime,npc_kill,solo_exp,party_exp,quest_exp,boss_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
0,2,12365,0.123246,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000
1,3,121046,2.732749,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.000000,2.419238e-03,0.000000
2,5,8885,1.012042,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000
3,7,37934,0.125616,0.033063,0.000000,0.321568,0.000000,1,0.000000,0.000000,0.0,0.000000,0.000000,7.580674e-05,0.000000
4,9,107257,0.033182,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000
5,13,111704,14.315530,8.487013,2.287007,0.000000,0.171112,0,1.717588,1.480916,0.0,9.309953,0.000000,-2.886437e+00,0.000000
6,14,44686,0.002370,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000
7,15,88391,0.040292,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000
8,17,111945,0.002370,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000
9,19,4250,0.357888,0.031713,0.013509,0.000000,0.066535,0,0.000000,0.000000,0.0,0.000000,0.000000,2.267965e-07,0.000000


#### activity 파일에 survival time을 입력
- 앞서 char_id와 acc_id를 기준으로 groupby 했던 파일과 label 파일을 acc_id 기준으로 merge
- 중복되는 값을 지니는 amount_spent를 삭제

In [5]:
label_new = pd.merge(activity_g, label, how = 'inner', on = 'acc_id').sort_values('char_id')
label_new.drop(columns = 'amount_spent', inplace = True)
# activity_new = label_new.set_index('char_id')


activity_new = label_new
# activity_new
# activity_new.columns
# z = activity_new[activity_new['char_id'] == 5] # char_id로 검색
# z

#### combat 파일을 groupby
- char_id, acc_id, class, level에 따라 groupby
- 이후 의미없는 'day' column 을 삭제

In [6]:
combat_1 = combat.groupby(['char_id', 'acc_id', 'class', 'level'],as_index=False).sum().sort_values('char_id')
# level이 달라지는 경우가 존재하기 때문에 level로는 묶지 않고
# 따로 구한 후 merge 혹은 join 시킨다

combat_1.drop(columns = 'day', inplace = True)


In [7]:
combat_1

,char_id,acc_id,class,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent
0,2,12365,2,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3,121046,3,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,5,8885,7,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,7,37934,2,14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,9,107257,7,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,13,111704,0,15,0.000000,0.000000,0.000000,1.439183,0.000000,0.225223,0.490644
6,14,44686,7,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,15,88391,6,14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,17,111945,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,19,4250,1,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### 중복되는 값들을 찾아서 확인
- char_id 가 같고 acc_id 가 다른 경우
- 수정할 필요가 있음
- combat에 있는 id 중에 activity에 없는 경우를 찾아 삭제 후 merge

In [8]:
# char_id 가 같고 acc_id가 다른 경우를 찾음
y = combat_1.groupby('char_id', as_index = False).count()
y
x = y[y['class']>2]
x
print(x['char_id'].values)
z = combat_1[combat_1['char_id'] == 82]
z
# level이 달라지는 경우 중복 값으로 표기된다.
# level이 달라지는 경우 최대값으로 계산

[    82    132    148 ... 397250 397402 397412]


,char_id,acc_id,class,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent
35,82,61018,1,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,82,61018,1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,82,61018,1,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# combat 에서 level이 최대값들로 행을 구한다.
cob_level = combat.groupby(['char_id', 'acc_id', 'class'], as_index = False).max()
cob_level

p = cob_level[cob_level['char_id'] == 82]
p

# char_id, acc_id level 을 추출
cob_level = cob_level[['char_id', 'acc_id', 'level']]
cob_level

# 149851 rows x 11 columns

,char_id,acc_id,level
0,2,12365,11
1,3,121046,11
2,5,8885,0
3,7,37934,14
4,9,107257,0
5,13,111704,15
6,14,44686,0
7,15,88391,14
8,17,111945,0
9,19,4250,6


In [10]:
# cob_level (캐릭터 lv) 과 combat 을 'char_id', 'acc_id', 'class'로
# groupby 한 combat_g를 merge

combat_g = combat.groupby(['char_id','acc_id','class'], as_index = False).sum()
combat_g.drop(columns = ['day', 'level'], inplace = True)
combat_g

combat_new = pd.merge(combat_g, cob_level, how = 'inner', on = ['char_id', 'acc_id'])
combat_new

,char_id,acc_id,class,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent,level
0,2,12365,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11
1,3,121046,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11
2,5,8885,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,7,37934,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14
4,9,107257,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
5,13,111704,0,0.000000,0.000000,0.000000,1.439183,0.000000,0.225223,0.490644,15
6,14,44686,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
7,15,88391,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14
8,17,111945,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
9,19,4250,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6


In [11]:
# char_id '5'을 검색
z = activity_new[activity_new['char_id'] == 5] # char_id로 검색
print(z)
print('='*80)
z = combat_new[combat_new['char_id'] == 5] # char_id로 검색
print(z)
print('='*80)
z = trade[trade['source_char_id'] == 5]
print(z)
print('='*80)
z = label[label['acc_id'] == 8885]
print(z)
print('='*80)
z = pledge[(pledge['acc_id'] == 8885) & (pledge['char_id'] == 5)]
print(z)

    char_id  acc_id  playtime  npc_kill  solo_exp  party_exp  quest_exp  \
15        5    8885  1.012042       0.0       0.0        0.0        0.0   

    boss_monster  death  revive  exp_recovery  fishing  private_shop  \
15             0    0.0     0.0           0.0      0.0           0.0   

    game_money_change  enchant_count  survival_time  
15                0.0            0.0             64  
   char_id  acc_id  class  pledge_cnt  random_attacker_cnt  \
2        5    8885      7         0.0                  0.0   

   random_defender_cnt  temp_cnt  same_pledge_cnt  etc_cnt  num_opponent  \
2                  0.0       0.0              0.0      0.0           0.0   

   level  
2      0  
Empty DataFrame
Columns: [day, time, type, server, source_acc_id, source_char_id, target_acc_id, target_char_id, item_type, item_amount, item_price]
Index: []
       acc_id  survival_time  amount_spent
38266    8885             64      0.098958
Empty DataFrame
Columns: [day, acc_id, char_id, ser

In [17]:
#### char_id 별 접속 일 수 확인
act_c = activity.groupby(['char_id','acc_id'], as_index = False).count()
act_c = act_c[['char_id','acc_id','day']]
act_c = act_c.rename(columns = {'day':'day_c'})

In [18]:
act_c['day_c'].max()

77